In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.ensemble import RandomForestClassifier

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Loading Data

In [2]:
# loading the train.csv data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
train_data.head() #DATA SAMPLE


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data Type & Information

In [3]:
train_data.info() 

# checking the data typee and the total number of non null vlaues

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Null Data Count

In [4]:
null_counts = train_data.isnull().sum()
print(null_counts)

#finding out the total number of null values in Data

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


# Male - Female | Survival Percentage (Full Data)

In [5]:
women = train_data.loc[train_data.Sex == 'female']['Survived']
rate_women = float(sum(women) / len(women))
print(f'Percentage of women who survided: {rate_women * 100:.1f}%')

Percentage of women who survided: 74.2%


In [6]:
male = train_data.loc[train_data.Sex == 'male']['Survived']
rate_men = float(sum(male) / len(male))
print(f'Percentage of men who survided: {rate_men * 100:.1f}%')

Percentage of men who survided: 18.9%


The code above calculates the percentage of male passengers (in train.csv) who survived.

From this you can see that almost 75% of the women on board survived, whereas only 19% of the men lived to tell about it. Since gender seems to be such a strong indicator of survival, the submission file in gender_submission.csv is not a bad first guess!



# Data Analysis (Full Data)

In [ ]:
import matplotlib.pyplot as plt # plotting figures


# specifies the parameters of our graphs
fig = plt.figure(figsize=(18,6), dpi=1600) 
alpha= alpha_scatterplot = 0.2  
alpha_bar_chart = 0.55 #color of the bar transmisivity 


plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.3, hspace=0.3) #adjusting the layout of the plots

# lets us plot many diffrent shaped graphs together 
ax1 = plt.subplot2grid((2,4),(0,0)) #space one
# plots a bar graph of those who surived vs those who did not.               
train_data.Survived.value_counts().plot(kind='bar', alpha=alpha_bar_chart)
# this nicely sets the margins in matplotlib to deal with a recent bug 1.3.1
ax1.set_xlim(-1, 2)
# puts a title on our graph
plt.title("Distribution of Survival, (1 = Survived)") 


# Create a subplot in a 2x3 grid starting at cell (0,1)
plt.subplot2grid((2,4),(0,1))
women = train_data[train_data.Sex == 'female']
plt.scatter(women.Survived, women.Age, alpha=alpha_scatterplot, color='red', label='Female')
male = train_data[train_data.Sex == 'male']
plt.scatter(male.Survived, male.Age, alpha=alpha_scatterplot, color='blue', label='Male')
# x, y, scatter
# sets the y axis lable
plt.ylabel("Age")
# formats the grid line style of our graphs                          
plt.grid(visible=True, which='major', axis='y')  #visible lines 
plt.title("Survival by Age,  (1 = Survived)")
plt.legend(loc='best')

# Plotting the Class Distrubution
ax3 = plt.subplot2grid((2,4),(0,2))
train_data.Pclass.value_counts().plot(kind="barh", alpha=alpha_bar_chart)
ax3.set_ylim(-1, len(train_data.Pclass.value_counts()))
plt.title("Class Distribution")

plt.subplot2grid((2,4),(1,0), colspan=2)
# plots class catigory change based on age density
train_data.Age[train_data.Pclass == 1].plot(kind='kde')    
train_data.Age[train_data.Pclass == 2].plot(kind='kde')
train_data.Age[train_data.Pclass == 3].plot(kind='kde')
plt.xlabel("Age") 
plt.title("Age Distribution within classes")
plt.legend(('1st Class', '2nd Class','3rd Class'),loc='best') 

#plotting passengers per boarding location
ax5 = plt.subplot2grid((2,4),(1,2))
train_data.Embarked.value_counts().plot(kind='bar', alpha=alpha_bar_chart)
ax1.set_xlim(-1, len(train_data.Embarked.value_counts()))
plt.title("Passengers per boarding locations")


plt.savefig("Data_With_Nan.png", dpi=300, bbox_inches='tight', pad_inches=0) #makes sure the output  of the data is constrained 


# Filtering Nan Data

In [ ]:
nonull_train_data = train_data.dropna()

nonull_train_data.info()

# Data Re-Analysis (Filtered Data)

In [9]:
import matplotlib.pyplot as plt # plotting figures


# specifies the parameters of our graphs
fig = plt.figure(figsize=(18,6), dpi=1600) 
alpha= alpha_scatterplot = 0.2  
alpha_bar_chart = 0.55 #color of the bar transmisivity 


plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.3, hspace=0.3) #adjusting the layout of the plots

# lets us plot many diffrent shaped graphs together 
ax1 = plt.subplot2grid((2,4),(0,0)) #space one
# plots a bar graph of those who surived vs those who did not.               
nonull_train_data.Survived.value_counts().plot(kind='bar', alpha=alpha_bar_chart)
# this nicely sets the margins in matplotlib to deal with a recent bug 1.3.1
ax1.set_xlim(-1, 2)
# puts a title on our graph
plt.title("Distribution of Survival, (1 = Survived)") 


# Create a subplot in a 2x3 grid starting at cell (0,1)
plt.subplot2grid((2,4),(0,1))
women = nonull_train_data[nonull_train_data.Sex == 'female']
plt.scatter(women.Survived, women.Age, alpha=alpha_scatterplot, color='red', label='Female')
male = nonull_train_data[nonull_train_data.Sex == 'male']
plt.scatter(male.Survived, male.Age, alpha=alpha_scatterplot, color='blue', label='Male')
# x, y, scatter
# sets the y axis lable
plt.ylabel("Age")
# formats the grid line style of our graphs                          
plt.grid(visible=True, which='major', axis='y')  #visible lines 
plt.title("Survival by Age,  (1 = Survived)")
plt.legend(loc='best')

# Plotting the Class Distrubution
ax3 = plt.subplot2grid((2,4),(0,2))
nonull_train_data.Pclass.value_counts().plot(kind="barh", alpha=alpha_bar_chart)
ax3.set_ylim(-1, len(nonull_train_data.Pclass.value_counts()))
plt.title("Class Distribution")

plt.subplot2grid((2,4),(1,0), colspan=2)
# plots class catigory change based on age density
nonull_train_data.Age[nonull_train_data.Pclass == 1].plot(kind='kde')    
nonull_train_data.Age[nonull_train_data.Pclass == 2].plot(kind='kde')
nonull_train_data.Age[nonull_train_data.Pclass == 3].plot(kind='kde')
plt.xlabel("Age") 
plt.title("Age Distribution within classes")
plt.legend(('1st Class', '2nd Class','3rd Class'),loc='best') 

#plotting passengers per boarding location
ax5 = plt.subplot2grid((2,4),(1,2))
nonull_train_data.Embarked.value_counts().plot(kind='bar', alpha=alpha_bar_chart)
ax1.set_xlim(-1, len(nonull_train_data.Embarked.value_counts()))
plt.title("Passengers per boarding locations")


plt.savefig("Data_Without_Nan.png", bbox_inches='tight', pad_inches=0)

# First Class Survival Analysis

In [10]:

# Percentage of people survived from first class 

classrate = train_data[(train_data['Pclass'] == 1)]
class_survive = float(classrate['Survived'].sum() / len(classrate))
print(f'Percentage of people in First Class who survided: {class_survive * 100:.1f}%')

Percentage of people in First Class who survided: 63.0%


In [11]:
classrate = train_data[(train_data['Pclass'] == 1) & (train_data['Sex'] == 'female')]
class_survive = float(classrate['Survived'].sum() / len(classrate))
print(f'Percentage of First Class Females who survived: {class_survive * 100:.1f}%')

# Total count of first class females
total_count = len(classrate)

# Count of survivors among first class females
survivor_count = classrate['Survived'].sum()

print(f'Total first class females: {total_count}')
print(f'Survivors among first class females: {survivor_count}')

Percentage of First Class Females who survived: 96.8%
Total first class females: 94
Survivors among first class females: 91


In [12]:
classrate = train_data[(train_data['Pclass'] == 1) & (train_data['Sex'] == 'male')]
class_survive = float(classrate['Survived'].sum() / len(classrate))
print(f'Percentage of First Class Males who survived: {class_survive * 100:.1f}%')

# Total count of first class males
total_count = len(classrate)

# Count of survivors among first class males
survivor_count = classrate['Survived'].sum()

print(f'Total first class males: {total_count}')
print(f'Survivors among first class males: {survivor_count}')

Percentage of First Class Males who survived: 36.9%
Total first class males: 122
Survivors among first class males: 45


# Second Class Survival Analysis

In [ ]:

# Percentage of people survived from first class 

classrate = train_data[(train_data['Pclass'] == 2)]
class_survive = float(classrate['Survived'].sum() / len(classrate))
print(f'Percentage of people in Second Class who survided: {class_survive * 100:.1f}%')

Percentage of people in Second Class who survided: 47.3%


In [ ]:
classrate = train_data[(train_data['Pclass'] == 2) & (train_data['Sex'] == 'female')]
class_survive = float(classrate['Survived'].sum() / len(classrate))
print(f'Percentage of Second Class Females who survived: {class_survive * 100:.1f}%')

# Total count of first class females
total_count = len(classrate)

# Count of survivors among first class females
survivor_count = classrate['Survived'].sum()

print(f'Total Second Class females: {total_count}')
print(f'Survivors among Second class females: {survivor_count}')

Percentage of Second Class Females who survived: 92.1%
Total Second Class females: 76
Survivors among Second class females: 70


In [ ]:
classrate = train_data[(train_data['Pclass'] == 2) & (train_data['Sex'] == 'male')]
class_survive = float(classrate['Survived'].sum() / len(classrate))
print(f'Percentage of Second Class Males who survived: {class_survive * 100:.1f}%')

# Total count of first class males
total_count = len(classrate)

# Count of survivors among first class males
survivor_count = classrate['Survived'].sum()

print(f'Total Second class males: {total_count}')
print(f'Survivors among Second class males: {survivor_count}')

Percentage of Second Class Males who survived: 15.7%
Total Second class males: 108
Survivors among Second class males: 17


# Prediction Model (Random Forest Classifier)

In [ ]:

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

print("Your submission was successfully saved!")


Your submission was successfully saved!
